<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 3 : Modularité</h1>

Le développement d'un grand programme demande une certaine organisation et, en particulier, un découpage des différents aspects du programme et des différentes tâches qui doivent être accomplies. Ceci est d'autant plus vrai lorsque plusieurs personnes participent au développement.  
Parmi ces questions, relevant du **génie logiciel**, on s'interesse à la manière dont les différentes parties d'un programme peuvent s'articuler.  
L'un des objectifs consiste à spécifier le rôle de chaque partie suffisamment précisément pour que chacune puisse ensuite être réalisé indépendamment des autres.

### Les ensembles
Le [paradoxe des anniversaires](https://images.math.cnrs.fr/Coincidences.html) nous permet de vérifier que dans un groupe (de, au moins, 23 personnes), il y a plus d'une chance sur deux pour que deux personnes aient leur anniversaire le même jour.  
On peut utiliser la fonction suivante avec des tableaux de 23 éléments (aléatoires) pour vérifier si un tableau contient un élément en double.

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = set()
    for elt in tab:
        if elt in ens:
            return True
        ens.add(elt)
    return False

On passe les élements en revue et on les stocke, à la volée, dans un ensemble `ens`, jusqu'à trouver un élément déjà présent dans cet ensemble, c'est-à-dire apparaissant déjà, plus tôt, dans le tableau.

In [ ]:
from random import randint

# Tests
# chaque anniversaire est un entier compris entre 1 et 366
cpt = 0
nb_simulations = 100
for _ in range(nb_simulations):
    classe = [randint(1, 366) for _ in range(23)] 
    if contient_doublon(classe):
        cpt += 1
print("La fréquence de classe avec 2 élèves ayant leur anniversaire à la même date est :",
      round(cpt / nb_simulations, 2))

Nous pouvons écrire une nouvelle version de cette fonction avec d'autres structures de données.

#### Deux approches rudimentaires
* Si nous souhaitons utiliser un tableau, en lieu et place d'un ensemble, cela va également fonctionner :

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = []
    for elt in tab:
        if elt in ens:
            return True
        ens.append(elt)
    return False

Le problème vient du test `elt in ens` dont le temps d'exécution est proportionnel au nombre d'élément stockés dans `ens`.  
On peut alors envisager une rechercher dichotomique mais cela implique que le tableau soit trié et l'opération `append` ne serait alors plus adaptée (il faudrait insérer l'élément au bon endroit et décaler tous les suivants). Le coût serait donc alors porportionnel au nombre d'éléments du tableau.
* Une autre approche serait d'utiliser un grand tableau de booléens (un pour chaque date possible) et on décide que `ens[elt]` vaut `True` si on a enregistré `elt` dans `tab` et `False` sinon : 

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = [False] * 367
    for elt in tab:
        if ens[elt]:
            return True
        ens[elt] = True
    return False

Ceci permet d'obtenir un test qui a un coût minimal mais nous contraint à utiliser une structure de données *volumineuse* pour un tableau, à priori, petit.

#### Représentation plus compacte : le tableau de bits
Les booléens occupent, en effet, beaucoup d'espace en mémoire en Python.

In [ ]:
from sys import getsizeof

print("L'espace mémoire pour le booléen", True, "est", getsizeof(True), "octets")
print("L'espace mémoire pour le booléen", False, "est", getsizeof(False), "octets")

Un seul bit est réellement utile pour choisir entre les deux valeurs `True` ou `False`.  
On peut donc essayer d'utiliser des entiers représentant plusieurs booléens et des opérations arithmétiques binaires.  
* Considérons un entier `ens` de 64 bits représentant un tableau `booleens` de 64 booléens, avec, comme clé de lecture que `booleens[elt]` vaut `True` si, et seulement si, le bit de rang `elt` de `ens` vaut `1`.  
Ainsi l'entier `26` qui s'écrit `00....0011010 ` représente le tableau de booléens `[False, True, False, True, True, True, False, False, ... , False]`, autrement dit l'ensemble $\{1,3,4\}$.  
* On peut alors utiliser l'entier $2^{elt}$, qui a tous ses bits à `0` sauf le bit de rang `elt` qui vaut `1`, comme révélateur.  
On calcule $2^{elt}$ avec l'opération de décalage `1 << elt`.
* En utilisant un **ou binaire** : `ens | (1 << elt)`, on fait passer le bit de rang `elt` de `ens` à `1` (et il y reste s'il y était déjà).  
En utilisant un **et binaire** : `ens & (1 << elt)`, on obtient soit $2^{elt}$ si `booleens[elt]` vaut `True`, soit $0$ sinon.  

On obtient alors :

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = 0
    for elt in tab:
        if ens & (1 << elt) != 0:
            return True
        ens = ens | (1 << elt)
    return False

En Python, la taille des entiers est illimitée. En pensant à d'autres langages de programmation (pour lesquels la taille des entiers est limitée (à 64 bits, par exemple)), on pourrait utiliser un tableau d'entiers ordinaires.  
Il suffit alors de décomposer `elt` sous la forme `elt = i + 64j` avec les opérations de quotient et de reste de la division entière et on fait correspondre à `elt` le bit de rang `i` de l'entier d'indice `j`

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = [0] * 6
    for elt in tab:
        if ens[elt // 64] & (1 << (elt % 64)) != 0:
            return True
        ens[elt // 64] = ens[elt // 64] | (1 << (elt % 64))
    return False

Le code s'exécute aussi vite et on gagne un facteur $64$ en occupation de la mémoire par rapport au tableau de booléens.  
Pour notre cas d'application, il suffit d'un tableau de six entiers pour représenter n'importe quel ensemble de dates d'anniversaires.

La représentation des ensembles développée dans les programmes précédents donne les idées essentielles de la structure de **tableau de bits**.  
Cette structure donne une représentation compacte d'un ensemble de booléens.  
Elle permet donc une meilleure utilisation des ressources de mémoire limitées comme la mémoire cache (mémoire accessible beaucoup plus rapidement que la mémoire vive de l'ordinateur et conservant temporairement les derniers éléments consultés).

#### Représentation plus souple : la table de hachage
Nous avons donc soit un *petit* tableau mais pour lequel chaque élément peut se trouver n'importe où, soit un *grand* tableau pour lequel il est immédiat de trouver la case du tableau liée à un élément donné. 

Nous pouvons essayer de combiner les avantages des deux méthodes : prenons un tableau de 23 cases (c'est le nombre d'éléments que l'on est susceptible de stocker dans notre ensemble) dans lequel chacune des 366 dates est liée, à priori, à une et une seule de ses cases.  
On peut alors enregistrer l'entier `elt` dans la case d'indice `elt % 23`.  
Il faut alors stocker, dans la case `ens[i]` un tableau contenant tous les éléments `elt` enregistrés tels que `elt % 23` vaut `i`.

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = [[] for _ in range(23)]
    for elt in tab:
        if elt in ens[elt % 23]:
            return True
        ens[elt % 23].append(elt)
    return False

Il y a un maximum de 23 éléments à répartir entre 23 petits tableaux, chacun d'entre eux sera quasiment vide : le coût des tests `elt in ens[elt % 23]` sera ainsi, en général, négligeable.  
Les jours de naissance étant, à peu près, également probables, la répartition dans les différents paquets a de grandes chances d'être équitable.

#### Factorisation du code
Dans chaque version de la fonction que nous avons construites, certaines parties n'ont jamais changé 

```python
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = ...
    for elt in tab:
        if ... :
            return True
        ...
    return False
```

Dans tous les cas, la structure `ens` représente un ensemble de dates.  
On pourrait donc isoler ces aspects dans des fonctions `cree`, `contient` et `ajoute`.

In [ ]:
def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = cree()
    for elt in tab:
        if contient(ens, elt):
            return True
        ajoute(ens, elt)
    return False

Ainsi, on pourrait changer la représentation de l'ensemble de date `ens` en changeant uniquement les définitions de ces trois fonctions, sans avoir à changer la fonction `contient_doublon`.  
Par ailleurs, la définition des ensembles de dates pourrait plus facilement être réutilisée dans un autre programme qui en aurait également besoin..

Ces trois fonctions définissent une **interface** entre un programme qui utilise une structure de données et un programme qui la définit. Celui qui l'utilise n'a pas besoin de connaître le détail de la réalisation de la structure utilisée, et cette réalisation peut, elle-même, être faite sans savoir comment, ni combien de fois, elle sera utilisée.

## Modules, interfaces et encapsulation
Une des clés du développement à grande échelle consiste à circonscrire et séparer proprement les différentes parties du programme.  
On peut, par exemple, séparer le code qui définit une structure de données, du code qui l'utilise, ou dans un plus grand projet séparer l'interface graphique du cœur de l'application.  
Chacun des morceaux obtenus peut être placé dans un fichier de code différent, appelé un [**module**](https://docs.python.org/fr/3/tutorial/modules.html).

Voici, par exemple, un petit module de manipulation de dates déjà vu qui sera défini par le fichier [`dates.py`](Fichiers/dates.py).

In [ ]:
def cree():
    """Crée et renvoie un ensemble de dates vide"""
    return [0] * 6

def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient la date val"""
    paquet = val // 64
    bit = val % 64
    return ens[paquet] & (1 << bit) != 0

def ajoute(ens, val) -> None:
    """Ajoute la date val à l'ensemble ens"""
    paquet = val // 64
    bit = val % 64
    ens[paquet] = ens[paquet] | (1 << bit)

### Découpage et réutilisation du code
Tout projet a, donc, un code séparé en plusieurs modules, les fonctionnalités définies dans l'un pouvant être utilisées par d'autres.  
Ainsi, une définition de la fonction `contient_doublon` utilisant le module `dates` serait le [suivant](Fichiers/doublon.py).

In [ ]:
from dates import cree, contient, ajoute

def contient_doublon(tab: list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = cree()
    for elt in tab:
        if contient(ens, elt):
            return True
        ajoute(ens, elt)
    return False

Un module peut donc faire référence à un certain nombre d'autres modules, qui peuvent aussi bien être issus du projet lui-même que de sources extérieures. On dit, alors, qu'il dépend des autres modules.  
En dehors de ces dépendances, un module reste, à priori, autonome et indépendant du reste du projet.

En conséquence, un module constitue également une brique logicielle pouvant être réutilisée dans d'autres projets (ou plusieurs fois dans un même projet).  
On a, donc, tout intérêt à ce que chaque module soit dédié à la résolution d'une tâche, ou d'un ensemble de tâches apparentées, clairement identifiable. Cela permet, notamment, à chaque fois que le même problème se pose à nouveau, de faire appel au module déjà défini plutôt que d'écrire une nouvelle fois un code similaire et essayer de résoudre un problème déjà résolu.

Ainsi, si l'on souhaite savoir combien d'élèves il faut, en moyenne, dans une école pour qu'un anniversaire soit fêté chaque jour, on peut utiliser un programme tirant des dates au hasard et les stockant dans un ensemble jusqu'à ce que toutes aient été obtenues au moins une fois.  
Le [programme suivant](Fichiers/fete.py) réalise cela en utilisant le module [`dates`](Fichiers/dates.py).


In [ ]:
from dates import cree, contient, ajoute
from random import randint

def fete_continue() -> int:
    """Renvoie le nombre d'élèves minimal pour que 2 élèves aient leur anniversaire le mëme jour"""
    compteur = 0
    nombre_dates = 0
    ens = cree()
    while nombre_dates < 366:
        compteur += 1
        val = randint(1, 366)
        if not contient(ens, val):
            nombre_dates += 1
            ajoute(ens, val)
    return compteur

n = 0
for _ in range(1000):
    n += fete_continue()
print("En moyenne", n / 1000, "élèves")

### Interfaces
Pour chaque module, on distingue :
* son **implémentation** (c'est-à-dire le code lui-même)
* son **interface**, constituant une énumération des fonctions définies dans le module qui sont destinées à être utilisées dans la réalisation d'autres modules, appelés **clients**

L'interface d'un module est liée à sa documentation, et doit notamment expliciter ce qu'un utilisateur a besoin de connaître des fonctions proposées : comment et pourquoi les utiliser.  
Pour chaque fonction de l'interface, on a ainsi besoin de son nom, de la liste de ses paramètres et de sa spécification (c'est-à-dire les conditions auxquelles la fonction peut être appliquée et les résultats à attendre).  
Des informations supplémentaires concernant des caractéristiques comme le temps d'exécution ou l'espace mémoire requis peuvent également être utiles.

L'objectif est que ces fonctions, incluses dans l'interface, soient suffisantes pour permettre à un utilisateur de faire appel aux fonctionnalités du module, et qu'elles puissent être utilisées sans avoir besoin d'aller consulter le code du module.  
L'interface peut être décrite comme une **abstraction** du module : une description de ce qui caractérise le module, mais faite à un niveau assez haut, ignorant les détails concrets de la réalisation.  
La documentation de l'interface peut être vu comme un **contrat** entre l'auteur d'un module et ses utilisateurs. Elle simplifie l'utilisation du module en limitant le nombre de choses qu'il faille lire, comprendre et mémoriser pour utiliser un module.

### Réalisation d'une interface
On dit qu'un module réalise une interface dès lors qu'il définit (au moins) toutes les fonctions promises par l'interface.

Voici un [programme](Fichiers/ensemble_v1.py), qui, comme [celui-ci](Fichiers/dates.py), réalise l'interface utilisée par la fonction [`contient_doublon`](Fichiers/doublon.py) :

In [ ]:
def cree():
    """Crée et renvoie un ensemble de dates vide"""
    return [[] for _ in range(23)]

def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient la date val"""
    return val in ens[val % 23]

def ajoute(ens, val) -> None:
    """ajoute la date val à l'ensemble ens"""
    ens[val % 23].append(val)
    
def enumere(ens) -> list:
    """Renvoie un tableau contenant les éléments de l'ensemble ens."""
    tab = []
    for paquet in ens:
        tab.extend(paquet)
    return tab

On a ajouté une fonction `enumere` construisant un tableau avec les éléments de l'ensemble `ens` donné en paramètre, que l'on peut utiliser, par exemple, pour écrire une boucle `for` sous la forme :

```python
for elt in enumere(ens):
```
pour énumérer les éléments de l'ensemble `ens`.

Ainsi, une même interface peut admettre une variété de réalisations radicalement différentes.

### Encapsulation
Le contrat qu'une interface établit entre le client et l'auteur d'un module ne porte pas sur les moyens, mais sur les résultats : l'auteur s'engage à ce que les résultats produits par l'utilisation de ses fonctions soient ceux décrits, mais il est libre de s'y prendre comme il le souhaite pour y parvenir.

Ainsi, l'auteur d'un module est libre d'utiliser les structures de données qui lui conviennent, mais aussi de définir toute une panoplie de fonctions ou objets annexes uniquement destinés à un usage interne et qui ne sont pas inclus dans l'interface.  
Ces éléments internes sont des *détails d'implémentation* mais peuvent constituer une large part du code et ne doivent pas être utilisés par les modules clients. Tous ces éléments, hors de l'interface, sont parfois qualifiés de **privés**, et on parle, à leur propos, d'**encapsulation** pour signifier qu'ils sont enfermés dans une boîte hermétique, dont l'utilisateur n'a pas à connaître le contenu et qu'il doit, encore moins, ouvrir.

Ce principe d'encapsulation réduit le couplage entre les différents modules, en évitant que les modifications du code d'un module ne nécessitent d'adapter le code de ses modules clients (pourvu que l'interface soit toujours respectée).  
Ainsi l'auteur d'un module peut mettre à jour son code pour le corriger ou l'améliorer, éventuellement en modifiant ou supprimant certains de ses éléments internes, sans que les projets dépendant de son module n'en souffrent : si ces derniers fonctionnaient avant la mise à jour, ils fonctionneront après.

On peut donc toujours modifier notre module `ensemble` pour l'adapter à la représentation d'entiers quelconques.  
Au lieu de constamment utiliser un tableau de 23 paquets qui deviendront tous très grands en moyenne si l'on doit stocker de nombreux éléments dans l'ensemble, on va permettre au nombre de paquets d'augmenter à mesure que le nombre d'éléments dans l'ensemble augmente aussi.

Le [programme](Fichiers/ensemble.py) suivant donne un tel programme dans lequel on commence avec 32 paquets, ce nombre étant doublé à chaque fois que le nombre d'éléments dépasse le nombre de paquets. Pour ne pas avoir besoin de recalculer en permanence la taille de l'ensemble représenté, on conserve, en outre cette information avec la table.  
Un ensemble est ainsi représenté par une paire nommée, réalisée par un dictionnaire contenant deux clés : `paquets` pour la table des paquets et `taille` pour le nombre d'éléments.

In [ ]:
def cree():
    """crée et renvoie un ensemble de valeurs vide"""
    return {'taille' : 0, 'paquets' : [[] for _ in range(32)]}

def contient(ens, val) -> bool:
    """renvoie True si, et seulement si, l'ensemble ens contient la valeur val"""
    p = val % len(ens['paquets'])
    return val in ens['paquets'][p]

def ajoute(ens, val) -> None:
    """ajoute la valeur val à l'ensemble ens"""
    if contient(ens, val):
        return None
    ens['taille'] += 1
    if ens['taille'] > len(ens['paquets']):
        _etend(ens)
    _ajoute_aux(ens['paquets'], val)
    
def _ajoute_aux(tab, val):
    p = val % len(tab)
    tab[p].append(val)
    
def _etend(ens):
    tmp = [[] for _ in range(2 * len(ens['paquets']))]
    for elt in enumere(ens):
        _ajoute_aux(tmp, elt)
    ens['paquets'] = tmp
    
def enumere(ens) -> list:
    """Renvoie un tableau contenant les éléments de l'ensemble ens."""
    tab = []
    for paquet in ens['paquets']:
        tab.extend(paquet)
    return tab

#### Encapsulation en Python
En Python, l'auteur d'un module peut indiquer que certains éléments (variables globales ou fonctions) sont privés en faisant commencer leur nom par le symbole `_`. Par convention, tous les autres éléments sont *publics* et doivent être compris comme appartenant à l'interface.

Cependant, en Python, l'encapsulation est une pure convention et son respect est sous la seule responsabilité des programmeurs des modules clients. Rien, dans les mécanismes du langage, n'empêche l'accès aux éléments privés, ni leur utilisation, ni leur modification.

De nombreux autres langages de programmation (comme Java, par exemple), mieux adaptés aux projets à grande échelle, introduisent, en revanche, un contrôle strict de l'encapsulation en rendant l'accès aux éléments privés très compliqué, voire impossible.

#### Table de hachage
La **table de hachage** est une structure de données fondamentale, sous-jacente aux [ensembles](https://docs.python.org/fr/3/tutorial/datastructures.html#sets) et aux [dictionnaires](https://docs.python.org/fr/3/faq/design.html#how-are-dictionaries-implemented-in-cpython) de Python.  
Cette structure, très polyvalente, permet de représenter des ensemble de taille arbitraire, avec des opérations d'accès aux éléments extrêmement rapides. Elle est considérée comme la plus efficace dans la plus grande variété des cas courants.

Le principal élément pour obtenir une table de hachage est une fonction appelée [**fonction de hachage**](https://docs.python.org/fr/3/library/functions.html#hash), qui prend, en paramètre, l'élément à stocker et renvoie un nombre entier définissant (modulo le nombre de paquets) le numéro du paquet dans lequel insérer l'élément.

Via cette fonction de codage de l'élément à stocker, on peut donc utiliser une table de hachage pour représenter des ensembles d'éléments de toutes sortes, et pas seulement des ensembles d'entiers.  

En outre, une fonction de hachage est souvent utilisée, même pour les tables stockant des entiers, afin de mélanger les éléments et éviter que des nombres liés les uns aux autres (par exemple, des multiples) ne se retrouvent systématiquement dans le même paquet.  
Dans le cas des nombres entiers, on pourra par exemple utiliser des multiplications par des grands nombres premiers combinées à des rotations de la représentation binaire. L'équité de la répartition des éléments dans les différents paquets, et donc l'efficacité de cette structure de données, dépend de la qualité du mélange opéré par cette fonction de hachage.  
Mais avec une fonction de hachage convenable, on peut, en pratique, considérer que la recherche ou l'ajout d'un élément dans une table de hachage sont des opérations instantanées.


In [ ]:
for elt in [123, 'numerique', 3.14, 123456789, 123456789123456789, 123456789123456789123456789]:
    print(hash(elt))

## Exceptions
Manipuler les fonctionnalités d'un module en n'utilisant que les fonctions fournies par son interface permet de ne pas se soucier des détails d'implémentation de ce module.

#### Erreurs
Considérons le module [`dates`](Fichiers/dates.py) et essayons d'ajouter, à un ensemble, une date invalide :

```python
>>> import dates
>>> s = dates.cree()
>>> dates.ajoute(s, 421)
Traceback (most recent call last):
  File "<pyshell>", line 1, in <module>
  File ".../dates.py", line 12, in ajoute
    ens[paquet] = ens[paquet] | (1 << bit)
IndexError: list index out of range
```

Le message :
* mentionne une [`IndexError`](https://docs.python.org/fr/3/library/exceptions.html#IndexError), alors que nous ne voulions pas savoir que cette structure utilisait, en interne, un tableau.
* ne permet, en rien, de comprendre le dépassement de tableau : il mentionne un tableau `ens` dont on ne connaît pas la longueur, et un indice `paquet` dont on ne connaît pas le lien avec les paramètres `s` et `421` fournis à `ajoute`.
Comprendre, en détail, cette erreur, demande d'explorer le code du module [`dates`](Fichiers/dates.py), ce qui fait perdre une partie des bénéfices de l'encapsulation.

D'ailleurs, d'autres dates invalides n'auraient pas déclencher d'erreur!

```python
>>> dates.ajoute(s, 377)
>>> dates.ajoute(s, -383)
```


En l'état actuel, une mauvaise utilisation des fonctions de l'interface risque d'engendrer des erreurs ou des effets collatéraux qui ne peuvent être compris et anticipés sans une connaissance approfondie du code de ces modules.  
Une meilleure pratique consiste, lors du développement des modules, à renvoyer à l'utilisateur des erreurs explicites, qui peuvent être interprétées à l'aide de la seule connaissance de l'interface.  
Cette pratique approfondit la notion de programmation défensive.

### Signaler un problème avec une exception
Il arrive, fréquemment, que les programmes s'interrompent avec des messages d'erreurs variés.

```python
>>> t = [1, 1, 2, 5, 14, 42, 132]
>>> t[12]
Traceback (most recent call last):
  File "<pyshell>", line 1, in <module>
IndexError: list index out of range
```

De telles erreurs sont appelées, en programmation, des [**exceptions**](https://docs.python.org/fr/3/tutorial/errors.html) et correspondent à la détection, faite par l'interprète Python, d'un problème empêchant la bonne exécution du programme.  
Lorsqu'une exception survient, l'exécution du programme est interrompue, à moins d'une prise en charge spécifique.

Voici quelques exceptions courantes, observables en utilisant les structures de base de Python :

| exception           | contexte                                        |
|---------------------|-------------------------------------------------|
| [`NameError`](https://docs.python.org/fr/3/library/exceptions.html#NameError)         | accès à une variable inexistante                |
| [`IndexError`](https://docs.python.org/fr/3/library/exceptions.html#IndexError)        | accès à un indice invalide d'un tableau         |
| [`KeyError`](https://docs.python.org/fr/3/library/exceptions.html#KeyError)          | accès à une clé inexistante d'un dictionnaire   |
| [`ZeroDivisionError`](https://docs.python.org/fr/3/library/exceptions.html#ZeroDivisionError) | division par zéro                               |
| [`TypeError`](https://docs.python.org/fr/3/library/exceptions.html#TypeError)         | opération appliquée à des valeurs incompatibles |

Il est possible de déclencher directement toutes ces exceptions (on dit [**lever une exception**](https://docs.python.org/fr/3/tutorial/errors.html#raising-exceptions)) avec l'opération [`raise`](https://docs.python.org/fr/3/reference/simple_stmts.html#the-raise-statement) de Python.

```python
>>> raise IndexError('indice trop grand')
Traceback (most recent call last):
  File "<pyshell>", line 1, in <module>
IndexError: indice trop grand
```

Cette opération s'écrit en faisant suivre le mot-clé `raise` du nom de l'exception à lever, lui même suivit, entre parenthèses, d'une chaîne de caractères donnant des informations sur l'erreur signalée.

On peut, ainsi, contrôler le flux d'exécution d'une fonction :

In [ ]:
def ecrit(tab, i, val):
    if i < 0:
        raise IndexError('indice négatif')
    tab[i] = val

La levée d'une exception avec `raise` interrompant l'éxecution du programme, nous n'avons pas besoin de `else`.

#### Observer la **pile d'appels**
Le message affiché lorsqu'une [exception](https://docs.python.org/fr/3/reference/executionmodel.html#exceptions) interrompt un programme donne un aperçu de l'état de la pile d'appels au moment où l'exception a été levée.  
On voit quelle première fonction a appelé quelle deuxième fonction qui, à son tour, ... jusqu'à arriver au point où l'exception a été levée.  
Il s'agit d'une information importante pour comprendre le contexte du problème et le corriger.

### Interfaces et exceptions
Les exceptions peuvent être utilisées, en particulier, dans les fonctions formant l'interface d'un module, pour signaler à un utilisateur du module toute utilisation incorrecte de ces fonctions.  
L'interface mentionnera, dans ce cas, quelles exceptions spécifiques sont levées et dans quelles conditions.

```python
def cree():
    """Crée et renvoie un ensemble vide"""
    
def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient l'élément val"""
    
def ajoute(ens, val) -> None:
    """Ajoute l'élément val à l'ensemble ens si i <= x <= 366 
    et lève une exception ValueError sinon"""
```

L'exception [`ValueError`](https://docs.python.org/fr/3/library/exceptions.html#ValueError) de Python est à utiliser dans les cas où un paramètre inadapté est donné à une fonction. C'est ici le cas lorsque le paramètre `val` de la fonction `ajoute` est un nombre qui n'est pas dans la plage représentant une date valide.

In [ ]:
def cree():
    """Crée et renvoie un ensemble vide"""
    return [0] * 6
    
def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient l'élément val"""
    if val < 1 or val > 366:
        return False
    paquet = val // 64
    bit = val % 64
    return ens[val] & (1 << bit) != 0
    
def ajoute(ens, val) -> None:
    """Ajoute l'élément val à l'ensemble ens si i <= x <= 366 
    et lève une exception ValueError sinon"""
    if val < 1 or val > 366:
        raise ValueError("date " + str(val) + " invalide")
    paquet = val // 64
    bit = val % 64
    ens[val] = ens[val] | (1 << bit)

### Rattraper une exception
Les exceptions levées par un programme peuvent avoir plusieurs causes.  
Certaines traduisent des erreurs du programme : elles sont imprévues et leurs conséquences ne sont, par définition, pas maîtrisées. Dans ces conditions, interrompre l'exécution du programme est légitime.  
D'autres exceptions, en revanche, s'inscrivent dans le fonctionnement normal du programme : elles correspondent à des situations connues, exceptionnelles mais possibles.

In [ ]:
date = int(input("Entrer un jour"))

Si la chaîne de caractères entrée par l'utilisateur ne représente pas un entier valide, la fonction `int` lève une exception `ValueError`.  
Une telle exception, qui correspond à un événement prévisible, n'est pas forcément fatale. Certes le programme ne peut pas poursuivre son exécution comme si de rien n'était, puisque certaines choses n'ont pas eu lieu normalement.  
Cependant, en tant que programmeur, on peut anticiper cette éventualité et prévoir un comportement alternatif du programme pour cette situation exceptionnelle.

Pour mettre en place ce comportement alternatif, il faut [**rattraper** cette exception](https://docs.python.org/fr/3/tutorial/errors.html#handling-exceptions), c'est-à-dire l'intercepter avant que l'exécution du programme ne soit définitivement abandonnée.

In [ ]:
try:
    date = int(input("Entrer un jour"))
except ValueError:
    print("Prière d'entrer un entier valide")

Le mot-clé [`try`](https://docs.python.org/fr/3/reference/compound_stmts.html#the-try-statement) introduit un premier bloc de code, puis le mot-clé [`except`](https://docs.python.org/fr/3/reference/compound_stmts.html#the-try-statement) suivi du nom de l'exception précède un deuxième bloc de code.

On exécute d'abord le bloc de code normal.
* Si l'exécution de ce bloc s'achève normalement, sans lever d'exception, alors le bloc alternatif est ignoré.
* Si, à l'inverse, une exception est levée dans l'exécution du code normal, alors l'exécution de ce bloc est immédiatement interrompue et le nom de l'exception levée est comparé avec le nom précisé à la ligne `except`.  
  * Si les noms correspondent, l'exception est **rattrapée** et on exécute le bloc de code alternatif.  
  * Sinon, l'exception est **propagée** et le programme s'interrompt.


In [ ]:
while True:
    try:
        date = int(input("Entrer un jour"))
        dates.ajoute(ens, date)
        break
    except ValueError:
        print("Il faut mettre un nombre entier compris entre 1 et 366")

#### Alternatives multiples
Si l'on prévoit que plusieurs exceptions peuvent être rattrapées, il est possible d'écrire plusieurs blocs alternatifs, chacun associé à sa ligne `except`.

```python
try:
    <bloc normal>
except Exception1:
    <bloc alternatif 1>
except Exception2:
    <bloc alternatif 2>
...    
```


#### Rattrapage indiscriminé
Si l'on ne précise pas de nom d'exception après `except`, alors toutes les exceptions seront rattrapées par ce bloc alternatif.

```python
try:
    <bloc normal>
except:
    <bloc alternatif>
```

C'est généralement une mauvaise idée, car on rattrape alors, à la fois, les exceptions qu'il est légitime de rattraper et celles qui traduisent des erreurs que l'on ne soupçonnait pas. Ce faisant, on efface donc les traces de ces éventuelles erreurs et on complique leur diagnostic.  

Un cas d'usage légitime du rattrapage indiscriminé consisterait à fournir quelques informations sur le contexte de l'exception, par un affichage ou une écriture dans un journal avant de relancer la propagation de l'exception. On peut réaliser ce redémarrage de la propagation en utilisant le mot-clé `raise` sans paramètre dans le bloc alternatif.

## Exercices
### Exercice 1
Ecrire un module réalisant l'interface 

```python
def cree():
    """Crée et renvoie un ensemble de dates vide"""

def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient la date val"""

def ajoute(ens, val) -> None:
    """Ajoute la date val à l'ensemble ens"""
```

suivant la stratégie du programme du cours :

*Si nous souhaitons utiliser un tableau ...*
```python
def contient_doublon(tab : list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = []
    for elt in tab:
        if elt in ens:
            return True
        ens.append(elt)
    return False
```


### Exercice 2
Ecrire un module réalisant l'interface 

```python
def cree():
    """Crée et renvoie un ensemble de dates vide"""

def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient la date val"""

def ajoute(ens, val) -> None:
    """Ajoute la date val à l'ensemble ens"""
```

suivant la stratégie du programme du cours :

*Si nous souhaitons utiliser un tableau de bits ...*
```python
def contient_doublon(tab : list) -> bool:
    """Renvoie True si le tableau contient un doublon. False sinon."""
    ens = 0
    for elt in tab:
        if ens & (1 << elt != 0:
            return True
        ens = ens | (1 << elt)
    return False
```

Attention : pour que la fonction `ajoute` fonctionne, il faut pouvoir modifier l'ensemble.

### Exercice 3
Supposons que l'on souhaite ajouter, à l'interface des ensembles :

```python
def cree():
    """Crée et renvoie un ensemble de dates vide"""

def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient la date val"""

def ajoute(ens, val) -> None:
    """Ajoute la date val à l'ensemble ens"""
```

la fonction `enumere` déjà présente dans le [programme](Fichiers/ensemble_v1.py) :

```python
def cree():
    """crée et renvoie un ensemble de dates vide"""
    return [[] for _ in range(23)]

def contient(ens, val) -> bool:
    """renvoie True si, et seulement si, l'ensemble ens contient la date val"""
    return val in ens[val % 23]

def ajoute(ens, val) -> None:
    """ajoute la date val à l'ensemble ens"""
    ens[val % 23].append(val)
    
def enumere(ens) -> list:
    """Renvoie un tableau contenant les éléments de l'ensemble ens."""
    tab = []
    for paquet in ens:
        tab.extend(paquet)
    return tab
```

Adapter le module du [programme suivant](Fichiers/dates.py) pour qu'il réalise l'interface étendue :
    
```python
def cree():
    """crée et renvoie un ensemble de dates vide"""
    return [0] * 6

def contient(ens, val) -> bool:
    """renvoie True si, et seulement si, l'ensemble ens contient la date val"""
    if val < 1 or val > 366:
        return False
    paquet = val // 64
    bit = val % 64
    return ens[paquet] & (1 << bit) != 0

def ajoute(ens, val) -> None:
    """ajoute la date val à l'ensemble ens"""
    if val < 1 or val > 366
        raise ValueError("date " + str(val) + " invalide")
    paquet = val // 64
    bit = val % 64
    ens[paquet] = ens[paquet] | (1 << bit)
```

### Exercice 4
Supposons que l'on souhaite ajouter, à l'interface des ensembles :

```python
def cree():
    """Crée et renvoie un ensemble de dates vide"""

def contient(ens, val) -> bool:
    """Renvoie True si, et seulement si, l'ensemble ens contient la date val"""

def ajoute(ens, val) -> None:
    """Ajoute la date val à l'ensemble ens"""
```

les deux fonctions d'union et d'intersection suivantes :

```python
def union(ens1, ens2):
    """Renvoie un nouvel ensemble composé de l'union des éléments de ens1 et ens2"""

def intersection(ens1, ens2):
    """Renvoie un nouvel ensemble composé de l'intersection des éléments de ens1 et ens2"""
```

Ces fonctions ne doivent pas modifier les ensembles `ens1` et `ens2` qui leur sont donnés en paramètres.

1. Réaliser ces fonctions dans le module de l'exercice 1.
2. Réaliser ces fonctions dans le module du [programme](Fichiers/dates.py).

### Exercice 5
Considérons une structure de table de hachage telle que réalisée par le [programme](Fichiers/ensemble.py) mais avec seulement 8 paquets.  
On place dedans les entiers 0, 1, 4, 9, 13, 24 et 30.  
Donner le contenu de chacun des paquets.

### Exercice 6
L'interface des tableaux de Python fournit de nombreuses opérations cachant une certaine complexité.  
Ecrire un module réalisant l'interface suivante :


```python
def tranche(tab, i, j):
    """Renvoie un nouveau tableau contenant les éléments de tab de l'indice i, inclus, 
    à l'indice j, exclu. Et le tableau vide si j <= i."""

def concaténation(tab1, tab2):
    """Renvoie un nouveau tableau contenant, dans l'ordre, 
    les éléments de tab1 puis les éléments de tab2"""
```

sans utiliser les opérations `+` et `t[i:j]`.  
Aucune de ces fonctions ne doit modifier les tableaux passés en paramètres.

### Exercice 7
Les tableaux de Python sont redimensionnables : leur nombre d'éléments peut augmenter au fil du temps.  
L'objectif de cet exercice est de définir un module réalisant une interface de tableau redimensionnable (sans utiliser les capacités de redimensionnement, natives des tableaux Python).

Voici une interface minimale pour une structure de tableau redimensionnable.

```python
def cree():
    """Crée et renvoie un tableau vide"""

def lit(tab, i):
    """Renvoie l'élément de tab à l'indice i"""
    
def ecrit(tab, i, val):
    """Place la valeur val dans la case d'indice i du tableau tab"""
    
def ajoute(tab, val):
    """Ajoute le nouvel élément val au tableau tab, après ses éléments actuels"""
```

Ces opérations sont équivalentes aux opérations `[]`, `tab[i]`, `tab[i] = val` et `tab.append(val)` des tableaux de python.

Pour réaliser cette interface, on va représenter un tableau redimensionnable `tab` de `n` éléments par une paire nommée contenant d'une part ce nombre `n` et d'autre part un tableau Python `t` dont la longueur est supérieure ou égale à `n`.  
Le nombre `n` sera appelé la **taille** de `tab` et la longueur de `t`, sa **capacité**.  
Les `n` éléments sont stockés dans les cases d'indices `0` à `n-1` du tableau `t`, tandis que les cases suivantes contiennent `None`.

1. Ecrire une fonction `cree()` créant et renvoyant un tableau redimensionnable de taille 0 et de capacité 8.
2. Ecrire deux fonctions `lit(tab, i)` et `ecrit(tab, i, val)` telles que décrites dans l'interface, en supposant que l'indice `i` fourni est bien compris entre `0` inclus et la taille de `tab` exclue.
3. Fonction `ajoute`.  
   a. Ecrire une fonction `_ajoute_aux(tab, val)` qui ajoute l'élément `val` à la fin du tableau redimensionnable `tab`, en supposant que la capacité de ce dernier est suffisante.  
   b. Ecrire une fonction `_double(tab)` qui double la capacité du tableau redimensionnable `tab`, en conservant ses éléments.  
   c. En déduire une fonction `ajoute(tab, val)` telle que décrite dans l'interface. Cette focntion doit doubler la capacité du tableau lorsqu'il ne peut pas accueillir de nouvel élément.
   


### Exercice 8
Voici une interface minimale pour une structure de dictionnaire.

```python
def cree():
    """Crée et renvoie un dictionnaire vide"""

def cle(dico, k):
    """Renvoie True si, et seulement si, le dictionnaire dico contient la clé k"""
    
def lit(dico, k):
    """Renvoie la valeur associée à la clé k dans le dictionnaire dico, 
    et None si la clé k n'apparait pas"""
    
def ecrit(dico, k, val):
    """Ajoute au dictionnaire dico l'association entre la clé k et la valeur val,
    en remplaçant une éventuelle association déjà présente pour k"""
```

On propose de réaliser cette interface de dictionnaire avec un tableau de couples clé-valeur, en faisant en sorte qu'aucune clé n'apparaisse deux fois.

1. Ecrire un module réalisant cela.
2. La description de l'une des quatre fonctions de notre interface ne correspond pas exactement à l'opération équivalente des dictionnaires de Python. Laquelle?  
Quelle expérience faire pour le confirmer?  
Corriger la description pour se rapprocher de celle de Python et adapter la réalisation.

## Liens :
* Document accompagnement Eduscol : [Modularité et API](https://eduscol.education.fr/document/7310/download)
* Documentation Python : [Erreurs et exceptions](https://docs.python.org/fr/3/tutorial/errors.html)
* Documentation Python : [Modules](https://docs.python.org/fr/3/tutorial/modules.html)
* Documentation Python - FAQ : [Modules](https://docs.python.org/fr/3/faq/programming.html#modules)
* Interstices : [La naissance du génie logiciel](https://interstices.info/la-naissance-du-genie-logiciel/)
* Interstices : [Le hachage](https://interstices.info/le-hachage/)
* Interstices : [50 ans d’interaction homme-machine](https://interstices.info/50-ans-dinteraction-homme-machine-retours-vers-le-futur/)
* Interstices : [Idée reçue : Une bonne interface, c’est une interface conviviale !](https://interstices.info/idee-recue-une-bonne-interface-cest-une-interface-conviviale/)
* [Complexité temporelle en Python](https://wiki.python.org/moin/TimeComplexity)